In [1]:
import os

import numpy as np
import pandas as pd
import sklearn.datasets as datasets

import revisor as rv

In [2]:
dataset = datasets.load_boston()
dataframe = pd.DataFrame(dataset.data, columns=dataset.feature_names)
dataframe['target'] = dataset.target

**IN JUPYTER NOTEBOOKS WE NEED TO WRAP THE IMPLEMENTATION CLASS**

In [3]:
def wrapper():
    class Implementation():
        def fit(self, model):
            from sklearn.neighbors import KNeighborsRegressor
            import pickle
            
            data = model.get_dataset()

            features = list(data.columns)
            features.remove('target')

            X = data[features]
            y = data['target']

            regressor = KNeighborsRegressor(
                n_neighbors = model.get_setting_value('neighbors')
            ).fit(X, y)

            pickle.dump(
                regressor,
                model.open_file_on_server('current.pickle', 'wb')
            )
            
            return regressor.score(X, y)
            
        def predict(self, model, X):
            import pickle
            import numpy as np
            
            regressor = pickle.load(
                model.open_file_on_server('current.pickle', 'rb')
            )

            return regressor.predict(np.array(X).reshape(
                -1, model.get_setting_value('shape')
            )).tolist()
    return Implementation

In [4]:
revisor_url = os.environ['REVISOR_SITE'] if 'REVISOR_SITE' in os.environ \
              else 'http://localhost:8000'
project = rv.Project('NotebookProject', revisor_url)
token = 'a5be1b7a9aa52b5470848fb0c84d715fd7b5963d' # admin's token

In [5]:
model = rv.Model('ExampleModel')
model.add_setting('neighbors', 'integer', 3)
model.add_setting('shape', 'integer', len(dataframe.columns) - 1)
model.set_dataset(dataframe)

In [6]:
assert project.deploy(model, wrapper, token)['ok']

In [7]:
import requests
scores = requests.post(
    revisor_url + '/projects/NotebookProject/fit',
    cookies={'token': token}
).json()
print("Score %.5f" % scores)

correct = dataframe['target'].values.tolist()
X = dataframe.drop('target', axis='columns').values.tolist()

predictions = requests.post(
    revisor_url + '/projects/NotebookProject/predict',
    cookies={'token': token},
    json=X
).json()

from sklearn.metrics import r2_score
print("Check %.5f" % r2_score(correct, predictions))

Score 0.78685
Check 0.78685
